In [15]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
from qiskit import Aer
from src.dnn import *
from src.data_encoders import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Post selection non-linearity

In [12]:
def non_linearity(x, weights, shots=1000):
    n = weights.shape[0]
    
    storage = qk.QuantumRegister(n, name="storage")
    clas_reg = qk.ClassicalRegister(2, name="clas_reg")
    registers = [storage, clas_reg]
    circuit = qk.QuantumCircuit(*registers)

    circuit.ry(x, storage[0])
    circuit.barrier()

    for i in range(n - 1):
        circuit.cx(storage[i], storage[i + 1])

    circuit.barrier()

    for i, w in enumerate(weights):
        circuit.ry(2*w, storage[i])

    circuit.barrier()    

    for i in range(n - 1):
        circuit.cx(storage[i], storage[i + 1])

    circuit.measure(storage[-2:], clas_reg)
    
    job = qk.execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts(circuit)
    
    states = ["00", "01", "10", "11"]
    
    for state in states:
        if state not in counts:
            counts[state] = 0
        
    return (counts["11"])/(counts["01"] + counts["11"]), (counts["01"] + counts["11"])/shots, circuit


def gradient(x, y, weights):
    n = weights.shape[0]
    gradient = np.zeros(weights.shape)
    
    for i in range(n): 
        A, B, _ = non_linearity(x, weights, shots=1000)
        weights[i] +=np.pi/4
        c, d, _ = non_linearity(x, weights, shots=1000)
        weights[i] -=np.pi/2
        e, f, _ = non_linearity(x, weights, shots=1000)
        weights[i] +=np.pi/4

        C = 0.5*(c-e)
        D = 0.5*(d-f)
        
        gradient = C - 0.5*A*D/B
        
        gradient = (A - y)*gradient

    
    return gradient
    

In [13]:
np.random.seed(42)
backend = Aer.get_backend('qasm_simulator')
n = 6
weights = np.random.uniform(0, np.pi, n)
x = np.array([0, 0.2, 0.4])
y = np.array([0.3, 0.7, 0.2])


In [14]:
grad = np.zeros(weights.shape)
m = x.shape[0]

for i in range(50):
    y_pred = []
    
    for x_, y_ in zip(x,y):
        grad += gradient(x_, y_, weights)
        y_pred.append(non_linearity(x_, weights, shots=10000)[0])
    
    grad = grad/m 
    
    weights += -0.1*grad
    print(y_pred)

[0.7825919507028692, 0.78123178550612, 0.7786932362521556]
[0.7727449455676516, 0.7694428037015161, 0.756385068762279]
[0.7572413793103449, 0.7502467917077986, 0.7509464036660689]
[0.7391478295659132, 0.7305751765893037, 0.7156181782099623]
[0.7196549599507086, 0.7151860134173612, 0.7047006830052229]
[0.6937936703658035, 0.7066258919469929, 0.6775111834078894]
[0.6836818368183681, 0.6798191533086724, 0.6779311765917989]
[0.6568205128205128, 0.6678430558438863, 0.6531075779475531]
[0.6437742001222743, 0.6475, 0.6375284149617689]
[0.6176590538336052, 0.634465820712239, 0.6069458631256384]
[0.614983305509182, 0.6023012552301256, 0.6033158107561666]
[0.5932446264073695, 0.5803625993073945, 0.5876288659793815]
[0.5888459189904336, 0.5772642871879513, 0.567890870933893]
[0.5606060606060606, 0.5530612244897959, 0.5606791029134354]
[0.5815284021250511, 0.5496417604912999, 0.5387589013224822]
[0.544884755357865, 0.5452880113983309, 0.5261669024045261]
[0.5377243066884176, 0.5245544554455446, 0.

In [6]:
y_pred, y_anc, circuit = non_linearity(0.3, weights, shots=10000)
print(y_pred, y_anc)
print(circuit)

0.4174994448145681 0.4503
            ┌─────────┐ ░                      ░ ┌────────────┐ ░           »
 storage_0: ┤ RY(0.3) ├─░───■──────────────────░─┤ RY(3.1179) ├─░───■───────»
            └─────────┘ ░ ┌─┴─┐                ░ ├────────────┤ ░ ┌─┴─┐     »
 storage_1: ────────────░─┤ X ├──■─────────────░─┤ RY(6.7381) ├─░─┤ X ├──■──»
                        ░ └───┘┌─┴─┐           ░ ├────────────┤ ░ └───┘┌─┴─┐»
 storage_2: ────────────░──────┤ X ├──■────────░─┤ RY(5.3638) ├─░──────┤ X ├»
                        ░      └───┘┌─┴─┐      ░ ├────────────┤ ░      └───┘»
 storage_3: ────────────░───────────┤ X ├──■───░─┤ RY(4.5261) ├─░───────────»
                        ░           └───┘┌─┴─┐ ░ ├────────────┤ ░           »
 storage_4: ────────────░────────────────┤ X ├─░─┤ RY(1.7449) ├─░───────────»
                        ░                └───┘ ░ └────────────┘ ░           »
clas_reg: 2/════════════════════════════════════════════════════════════════»
                                      